<a href="https://colab.research.google.com/github/n00blet/OpenNMT-Machine-Translation/blob/master/notebook/NeuralMachineTranslation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Neural Machine Translation using Seq2Seq Encoder-Decoder**

\\



**Description**

Language translation has been an open problem for many years now. Even though there are many tools like Google translate, DeepL they are still evolving and far near perfect to human translation. And languages in general are vast, as in a statement can be expressed or written in more than one way. In this tutorial we will try to build a simple and basic machine translation system using [OpenNMT](http://opennmt.net/).


\\



**Pre-Requisite **

This article assumes that you have some basic knowledge in Python programming and Neural Machine Translation.

\\

**Requirements**

*  Python Virtual Environment (Recommended)

*  [TensorFlow](https://www.tensorflow.org/install) (GPU version)

*   [CUDA](https://developer.nvidia.com/cuda-downloads) compatible Nvidia Graphics Card

* [OpenNMT](https://github.com/OpenNMT/OpenNMT-tf)


*   [Sentence Piece](https://github.com/google/sentencepiece#c-from-source)(Recommended Installation from Source)






\\

**Overview**


*   PreProcessing the Data
*   Training with different hyperparameters
*   Inference or Translation

\\

**Diving into OpenNMT**

For the tasks further below, we will download one of the dataset from Machine Translation Research websites [WMT Translation Task](http://www.statmt.org/wmt16/translation-task.html)  or  [OPUS Parallel Corpus](http://opus.nlpl.eu/) .




\\

**Step 1: Preprocessing**

Assuming that we have our environment ready and configured for training using GPU, the first step in this task is to preprocess the raw data.

Here we build **source** and **target** word vocabularies using an Unsupervised Text Tokenizer [SentencePiece](https://github.com/google/sentencepiece). 

Before we go into the next task,









\\

**Step 2: Training  and Evaluating  the data**



**Step 3: Traslation and Testing the results**